In [2]:
#nuevo enfoque
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

import math

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [3]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

In [7]:
from google.cloud import storage

def listar_objetos_en_carpeta(nombre_bucket: str, nombre_carpeta: str) -> list[str]:
    """
    Lista los nombres de todos los objetos (archivos) dentro de una carpeta específica
    en un bucket de Google Cloud Storage.

    Args:
        nombre_bucket: El nombre de tu bucket de GCS (ej. 'vboxiooof').
        nombre_carpeta: La ruta de la carpeta dentro del bucket (ej. 'videos/').

    Returns:
        Una lista de strings con los nombres completos de los objetos.
    """
    # Inicializa el cliente de GCS. Asegúrate de que tus credenciales
    # estén configuradas en el entorno donde ejecutes el código.
    storage_client = storage.Client()
    
    # Obtiene el objeto bucket.
    bucket = storage_client.get_bucket(nombre_bucket)

    # Usa el parámetro 'prefix' para filtrar los objetos por la carpeta.
    # El 'prefix' es la ruta de la carpeta, por ejemplo, 'videos/'.
    # Si quieres listar solo los archivos de video, puedes agregar un filtro adicional.
    blobs = bucket.list_blobs(prefix=nombre_carpeta)

    nombres_objetos = []
    print(f"Buscando objetos en la carpeta '{nombre_carpeta}' del bucket '{nombre_bucket}'...")
    
    for blob in blobs:
        # Aquí, 'blob.name' te da la ruta completa del objeto, como 'videos/mi_video.mp4'.
        # Puedes añadir una condición para ignorar la propia carpeta si está listada.
        if blob.name != nombre_carpeta:
            nombres_objetos.append(blob.name)
            print(f"  - Objeto encontrado: {blob.name}")
            
    print(f"\nSe encontraron {len(nombres_objetos)} objetos.")
    return nombres_objetos

# --- Ejemplo de uso ---
if __name__ == "__main__":
    tu_bucket = "vboxiooof"
    tu_carpeta = "Videos/Videos_Segmentados/"  # Asegúrate de incluir la barra diagonal al final

    # Llama a la función para obtener la lista de objetos.
    lista_de_archivos = listar_objetos_en_carpeta(tu_bucket, tu_carpeta)

    # Puedes imprimir la lista completa si lo deseas
    # print("\nLista completa de archivos:", lista_de_archivos)
    
    # Esta lista de archivos la puedes usar para generar las URIs
    # 'gs://bucket_name/object_name' para enviárselas a Gemini.

Buscando objetos en la carpeta 'Videos/Videos_Segmentados/' del bucket 'vboxiooof'...
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_000.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_001.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_002.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_003.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_004.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_005.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_006.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_007.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_008.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_009.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_010.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_011.mkv
  - Objeto enc

In [8]:
from google.cloud import storage
from vertexai.preview.generative_models import GenerativeModel, Part
import time
from typing import List, Dict

def analizar_videos_con_gemini(
    nombre_bucket: str, 
    nombres_videos: List[str], 
    pregunta_usuario: str,
    max_retries: int = 3,
    delay: float = 5.0
) -> Dict[str, str]:
    """
    Analiza una lista de videos con el modelo Gemini según la pregunta del usuario.
    
    Args:
        nombre_bucket: Nombre del bucket de GCS
        nombres_videos: Lista de rutas a los videos en el bucket
        pregunta_usuario: Pregunta que el usuario quiere hacer sobre los videos
        max_retries: Intentos máximos por video en caso de error
        delay: Tiempo de espera entre reintentos
        
    Returns:
        Diccionario con los resultados por video: {nombre_video: respuesta_gemini}
    """
    # Inicializar el modelo Gemini
    model = GenerativeModel('gemini-2.5-flash-lite-preview-06-17')
    
    # Diccionario para almacenar resultados
    resultados = {}
    
    # Crear URI GCS para cada video
    for video_path in nombres_videos:
        gcs_uri = f"gs://{nombre_bucket}/{video_path}"
        print(f"\nAnalizando video: {video_path}")
        
        # Configurar el prompt con contexto
        prompt = f"""
        Eres un analista experto en videos. El usuario te hará preguntas sobre el contenido visual.
        Video a analizar: {gcs_uri}
        Pregunta del usuario: {pregunta_usuario}
        
        Proporciona una respuesta detallada basada únicamente en lo que puedes observar en el video.
        Si no puedes determinar la respuesta, indica claramente por qué no es posible responder.
        """
        
        # Intentar el análisis con reintentos
        for attempt in range(max_retries):
            try:
                # Enviar el video y la pregunta a Gemini
                response = model.generate_content(
                    [
                        Part.from_uri(gcs_uri, mime_type="video/mp4"),
                        prompt
                    ]
                )
                
                # Almacenar la respuesta
                resultados[video_path] = response.text
                print(f"Análisis completado para {video_path}")
                break  # Salir del bucle de reintentos si tiene éxito
                
            except Exception as e:
                print(f"Intento {attempt + 1} fallido para {video_path}: {str(e)}")
                if attempt == max_retries - 1:
                    resultados[video_path] = f"Error al analizar: {str(e)}"
                else:
                    time.sleep(delay)  # Esperar antes de reintentar
    
    return resultados


# --- Función principal que integra todo ---
def analizar_videos_en_bucket(
    nombre_bucket: str, 
    carpeta_videos: str, 
    pregunta_usuario: str
) -> Dict[str, str]:
    """
    Función completa que lista videos en un bucket y los analiza con Gemini.
    
    Args:
        nombre_bucket: Nombre del bucket de GCS
        carpeta_videos: Ruta de la carpeta con videos
        pregunta_usuario: Pregunta para analizar los videos
        
    Returns:
        Resultados del análisis por video
    """
    # 1. Listar los videos en el bucket
    print("Listando videos en el bucket...")
    videos = listar_objetos_en_carpeta(nombre_bucket, carpeta_videos)
    
    # Filtrar solo archivos de video (opcional)
    video_extensions = ('.mp4', '.mov', '.avi', '.mkv')
    videos = [v for v in videos if v.lower().endswith(video_extensions)]
    
    if not videos:
        print("No se encontraron videos para analizar.")
        return {}
    
    print(f"\nSe analizarán {len(videos)} videos con Gemini...")
    
    # 2. Analizar cada video con Gemini
    resultados = analizar_videos_con_gemini(nombre_bucket, videos, pregunta_usuario)
    
    return resultados


# --- Ejemplo de uso ---
if __name__ == "__main__":
    # Configuración
    BUCKET = "vboxiooof"
    CARPETA = "Videos/Videos_Segmentados/"
    PREGUNTA = "¿Qué marcas o logos comerciales son visibles en este video? Por favor, lista todas las que puedas identificar y menciona en qué momento aparecen."
    
    # Ejecutar análisis completo
    resultados = analizar_videos_en_bucket(BUCKET, CARPETA, PREGUNTA)
    
    # Mostrar resultados
    print("\n\n--- Resultados del análisis ---")
    for video, respuesta in resultados.items():
        print(f"\nVideo: {video}")
        print(f"Respuesta Gemini:\n{respuesta}\n")
        print("-" * 50)

Listando videos en el bucket...
Buscando objetos en la carpeta 'Videos/Videos_Segmentados/' del bucket 'vboxiooof'...
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_000.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_001.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_002.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_003.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_004.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_005.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_006.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_007.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_008.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_009.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_010.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicost

/Users/wivboost/Documents/Repositorio/Embeddings_OFF/mi-entorno/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()



Analizando video: Videos/Videos_Segmentados/mexicosta_segment_000.mkv
Intento 1 fallido para Videos/Videos_Segmentados/mexicosta_segment_000.mkv: 404 Publisher Model `projects/constant-setup-463820-p6/locations/us-central1/publishers/google/models/gemini-2.5-flash-lite-preview-06-17` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions
Intento 2 fallido para Videos/Videos_Segmentados/mexicosta_segment_000.mkv: 404 Publisher Model `projects/constant-setup-463820-p6/locations/us-central1/publishers/google/models/gemini-2.5-flash-lite-preview-06-17` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions


KeyboardInterrupt: 

In [14]:
# --- Inicialización del LLM ---
model = GenerativeModel('gemini-2.0-flash-lite') # Debo probar con varios modelos, para ver cual tiene mejor rendimiento.



In [15]:
from google.cloud import storage
from vertexai.preview.generative_models import GenerativeModel, Part
import time
from typing import List, Dict

def analizar_primeros_5_videos(
    nombre_bucket: str, 
    carpeta_videos: str, 
    pregunta_usuario: str
) -> Dict[str, str]:
    """
    Analiza solo los primeros 5 videos de un bucket con Gemini.
    
    Args:
        nombre_bucket: Nombre del bucket de GCS
        carpeta_videos: Ruta de la carpeta con videos
        pregunta_usuario: Pregunta para analizar los videos
        
    Returns:
        Resultados del análisis por video (solo primeros 5)
    """
    # 1. Listar los videos en el bucket
    print("Listando videos en el bucket...")
    todos_videos = listar_objetos_en_carpeta(nombre_bucket, carpeta_videos)
    
    # Filtrar solo archivos de video y tomar primeros 5
    video_extensions = ('.mp4', '.mov', '.avi', '.mkv')
    videos = [v for v in todos_videos if v.lower().endswith(video_extensions)][:5]
    
    if not videos:
        print("No se encontraron videos para analizar.")
        return {}
    
    print(f"\nSe analizarán los primeros {len(videos)} videos con Gemini...")
    for i, video in enumerate(videos, 1):
        print(f"{i}. {video}")
    
    # 2. Analizar cada video con Gemini
    resultados = analizar_videos_con_gemini(nombre_bucket, videos, pregunta_usuario)
    
    return resultados


# --- Función de análisis de videos (la misma que antes) ---
def analizar_videos_con_gemini(
    nombre_bucket: str, 
    nombres_videos: List[str], 
    pregunta_usuario: str,
    max_retries: int = 3,
    delay: float = 5.0
) -> Dict[str, str]:
    """
    Analiza una lista de videos con el modelo Gemini según la pregunta del usuario.
    """
    model = GenerativeModel('gemini-2.5-flash-lite-preview-06-17')
    resultados = {}
    
    for video_path in nombres_videos:
        gcs_uri = f"gs://{nombre_bucket}/{video_path}"
        print(f"\nAnalizando video: {video_path}")
        
        prompt = f"""
        Eres un analista experto en videos. El usuario te hará preguntas sobre el contenido visual.
        Video a analizar: {gcs_uri}
        Pregunta del usuario: {pregunta_usuario}
        
        Proporciona una respuesta detallada basada únicamente en lo que puedes observar en el video.
        Si no puedes determinar la respuesta, indica claramente por qué no es posible responder.
        """
        
        for attempt in range(max_retries):
            try:
                response = model.generate_content(
                    [
                        Part.from_uri(gcs_uri, mime_type="video/mp4"),
                        prompt
                    ]
                )
                resultados[video_path] = response.text
                print(f"Análisis completado para {video_path}")
                break
                
            except Exception as e:
                print(f"Intento {attempt + 1} fallido para {video_path}: {str(e)}")
                if attempt == max_retries - 1:
                    resultados[video_path] = f"Error al analizar: {str(e)}"
                else:
                    time.sleep(delay)
    
    return resultados


# --- Ejemplo de uso ---
if __name__ == "__main__":
    # Configuración
    BUCKET = "vboxiooof"
    CARPETA = "Videos/Videos_Segmentados/"
    PREGUNTA = "¿Qué marcas o logos comerciales son visibles en este video? Por favor, lista todas las que puedas identificar."
    
    # Ejecutar análisis solo para primeros 5 videos
    resultados = analizar_primeros_5_videos(BUCKET, CARPETA, PREGUNTA)
    
    # Mostrar resultados
    print("\n\n--- Resultados del análisis (primeros 5 videos) ---")
    for video, respuesta in resultados.items():
        print(f"\nVideo: {video}")
        print(f"Respuesta Gemini:\n{respuesta}\n")
        print("-" * 50)

Listando videos en el bucket...
Buscando objetos en la carpeta 'Videos/Videos_Segmentados/' del bucket 'vboxiooof'...
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_000.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_001.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_002.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_003.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_004.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_005.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_006.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_007.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_008.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_009.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicosta_segment_010.mkv
  - Objeto encontrado: Videos/Videos_Segmentados/mexicost

/Users/wivboost/Documents/Repositorio/Embeddings_OFF/mi-entorno/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Intento 1 fallido para Videos/Videos_Segmentados/mexicosta_segment_000.mkv: 404 Publisher Model `projects/constant-setup-463820-p6/locations/us-central1/publishers/google/models/gemini-2.5-flash-lite-preview-06-17` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions
Intento 2 fallido para Videos/Videos_Segmentados/mexicosta_segment_000.mkv: 404 Publisher Model `projects/constant-setup-463820-p6/locations/us-central1/publishers/google/models/gemini-2.5-flash-lite-preview-06-17` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions
Intento 3 fallido para Videos/Videos_Segmentados/mexicosta_segment_000.mkv: 404 Publisher Model `projects/constant-setup-463820-p6/locations/us-cent

KeyboardInterrupt: 